<a href="https://colab.research.google.com/github/saritdi/PlantClassification/blob/main/Sarit_My_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
start_time = datetime.now()
print("--- {} ---",start_time)


In [ ]:
# Stage 1: Install dependencies and setting up GPU environment
!pip install tensorflow-gpu==2.3.0

In [ ]:
# Importing a library that is not in Colaboratory
!apt-get -qq install -y libfluidsynth1
!pip install matplotlib-venn

In [ ]:
!pip install tqdm

In [ ]:
# Dataset preprocessing
# Import project dependencies
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


%matplotlib inline
tf.__version__

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

In [ ]:
# Authenticate and create the PyDrive client.
drive.mount('/content/gdrive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
driver = GoogleDrive(gauth)


In [ ]:
# Setting up data import for google Drive
# Training data
fid = driver.ListFile({'q':"title='Train102.zip'"}).GetList()[0]['id']
f = driver.CreateFile({'id': fid})
f.GetContentFile('Train102.zip')

# Testing data
fid1 = driver.ListFile({'q':"title='Test102.zip'"}).GetList()[0]['id']
f1 = driver.CreateFile({'id': fid1})
f1.GetContentFile('Test102.zip')

fid2 = driver.ListFile({'q':"title='MyHousePlants.zip'"}).GetList()[0]['id']
f2 = driver.CreateFile({'id': fid2})
f2.GetContentFile('MyHousePlants.zip')

# Testing data
fid3 = driver.ListFile({'q':"title='Test.zip'"}).GetList()[0]['id']
f3 = driver.CreateFile({'id': fid3})
f3.GetContentFile('Test.zip')





In [ ]:
f.keys()

In [ ]:
f1.keys()

In [ ]:
f2.keys()

In [ ]:
f3.keys()

In [ ]:
!unzip Train102.zip

In [ ]:
!unzip MyHousePlants.zip

In [ ]:
!unzip Test102.zip 

In [ ]:
!unzip Test.zip 

In [ ]:
PATH = '/content'
os.listdir(PATH)

In [ ]:
train_data_dir_102 = os.path.join(PATH,'Train102')
test_data_dir_102 = os.path.join(PATH,'Test102')
train_data_dir = os.path.join(PATH,'MyHousePlants')
test_data_dir = os.path.join(PATH,'Test')


In [ ]:
print(os.listdir(test_data_dir))

['Sansevieria Ballyi', 'Parlor Palm', 'Coleus', 'House Leek', 'Paddle Plant', 'Nerve Plant', 'Zebra Cactus', 'Moon Cactus', 'Venus Fly Trap', 'String Of Banana', 'Lucky Bamboo', 'Poinsettia', "Elephant's Ear", 'Begonia Maculata', 'Jade Plant']


In [ ]:
# Preparing the data
IMG_HEIGHT = 256
IMG_WIDTH = 256

# # Image normalization
# Image Data Augmentation
# # 255.0 is the highest pixel value
image_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1,
    horizontal_flip = True,
    vertical_flip = True, 
    rotation_range = 90,
    brightness_range = [0.5,1.0]
    )

train_data_generator_102 = image_generator.flow_from_directory(
    directory=train_data_dir_102,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    subset="training",
    class_mode='sparse')

valid_data_generator_102 = image_generator.flow_from_directory(
    directory=train_data_dir_102,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    subset="validation",
    class_mode='sparse')

test_data_generator_102 = image_generator.flow_from_directory(
    directory=test_data_dir_102,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'sparse')

train_data_generator = image_generator.flow_from_directory(
    directory=train_data_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    subset="training",
    class_mode='sparse')

valid_data_generator = image_generator.flow_from_directory(
    directory=train_data_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    subset="validation",
    class_mode='sparse')

test_data_generator = image_generator.flow_from_directory(
    directory=test_data_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'sparse')

class_names = list(train_data_generator.class_indices.keys())

In [ ]:
# This function will plot images in the form of a grid with 1 row and 3 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 3, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

sample_training_images, _ = next(train_data_generator_102)
plotImages(sample_training_images[:3])

In [ ]:
# Loading the pre-trained model (MobileNetV2)
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
base_model = tf.keras.applications.DenseNet201(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')


In [ ]:
# Freezing the base model
# base_model.shape=(None, 4, 4, 1280) - too big
base_model.trainable = False
print(base_model.output)
# takes the whole input instead of taking parts at a time
# takes an average of numbers in an input = output of the base_model
# convert features to vectors
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
print(global_average_layer)
prediction_layer = tf.keras.layers.Dense(units=102, activation='softmax')(global_average_layer)
print(prediction_layer)

In [ ]:
# Defining the model
# Combining two models
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="sparse_categorical_crossentropy", metrics=["SparseCategoricalAccuracy"])


In [ ]:
train = train_data_generator_102
valid = valid_data_generator_102
epochs = 50
history = model.fit(train,  
                    epochs = epochs, 
                    validation_data=valid)

In [ ]:
import plotly.graph_objects as go

acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
valid_loss, valid_accuracy = model.evaluate(valid)

In [ ]:
print("Accuracy after transfer learnןing: {}".format(valid_accuracy))

In [ ]:
test_accu = model.evaluate(test_data_generator_102)
print('The testing accuracy is :',test_accu[1]*100, '%')




In [ ]:
base_model.trainable = True

In [ ]:
#fine_tune_at = 100

In [ ]:
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False


In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['SparseCategoricalAccuracy'])
#model.summary()

In [ ]:

train = train_data_generator
valid = valid_data_generator
epochs = 30
history = model.fit(train,  
                    epochs = epochs, 
                    validation_data=valid)

In [ ]:
valid_loss, valid_accuracy = model.evaluate(valid)

In [ ]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

In [ ]:
import plotly.graph_objects as go

acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_accu = model.evaluate(test_data_generator)
print('The testing accuracy is :',test_accu[1]*100, '%')

In [ ]:
end_time = datetime.now()
print("--- {} ---",end_time)
print("-----total time {}-----",end_time-start_time)


In [ ]:
### Yet to be implemented

# Save my model
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

In [ ]:
# Examine your saved model
!saved_model_cli show --dir {export_path} --all

In [ ]:
# Serve your model with TensorFlow Serving

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [ ]:
# Install TensorFlow Serving
!apt-get install tensorflow-model-server

### Start running TensorFlow Serving

This is where we start running TensorFlow Serving and load our model.  After it loads we can start making inference requests using REST.  There are some important parameters:

* `rest_api_port`: The port that you'll use for REST requests.
* `model_name`: You'll use this in the URL of REST requests.  It can be anything.
* `model_base_path`: This is the path to the directory where you've saved your model.


In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=PlantIL_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


In [ ]:
!tail server.log

## Make a request to your model in TensorFlow Serving

This part of the code is still not implemented


In [ ]:
test = test_data_generator
test_images, test_labels  = next(test)
test_labels = [int(i) for i in list(test_labels)]
class_names = list(test_data_generator.class_indices.keys())
print(class_names)

def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx])
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))

In [ ]:
import json
test = test_data_generator
test_images, test_labels  = next(test)
test_labels = [int(i) for i in list(test_labels)]
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:10].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/PlantIL_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']
print(predictions[0])
print(np.argmax(predictions[0]))

show(0, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
  class_names[np.argmax(predictions[0])], np.argmax(predictions[0]), class_names[test_labels[0]], test_labels[0]))

In [ ]:
model.save('/content/UrbanPlantClassifier.h5')
model.save('/content/gdrive/MyDrive/UrbanPlantClassifier.h5')




In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/gdrive/MyDrive/DenseNet201-4.h5')
print(model)